This notebook shows how to run the three functions I made to condense the important steps from the given class notebooks.

The goal is to abstract as much behind-the-scenes functionality as possible to make the image processing pipeline simple and easy to run

**The only configuration you must set is your astrometry.net API key. Set this in the /src/content/pipeline/config/dconf_dah.txt file**

Besides that, this does not require any setup or configurations in any other files, though default path values are set in setup.py. It is optional to set these in setup.py, as you can also set all paths from function calls alone.

In [ ]:
# Install dependencies
!pip3 install -r requirements.txt

In [55]:
from image_math import batch_process
from setup import *
from pipeline import run_pipeline, run_local_astrometry
from image_combine import drizzle, display_images, datain
from photutils import detect_sources
out = '/Users/jonahdf/Local Documents/Astro/M82 1:28/m82-cosmic-test'

In [8]:
"""
Batch_process
    Batch process HDR images.
    Inputs:
        datapaths (optional list str): list of paths to data files (default: datapath from input.py)
        outfolder (optional str): path to output folder (default: outpath from input.py)
        darkfolder (optional str): path to dark folder (default: darkpath from input.py)
        flatfolder (optional str): path to flat folder (default: flatpath from input.py)
        biasfolder (optional str): path to bias folder (default: biaspath from input.py)
    Outputs:
        Saves HDR images to output path.
""" 
darks = '/Users/jonahdf/Local Documents/Astro/M82 1:28/darks'
flats = '/Users/jonahdf/Local Documents/Astro/M82 1:28/flats'
bias = '/Users/jonahdf/Local Documents/Astro/M82 1:28/bias'
infolders = ['/Users/jonahdf/Local Documents/Astro/M82 1:28/h-alpha']
            
batch_process(datapaths = infolders, outfolder = out, darkfolder = darks, flatfolder = flats, biasfolder = bias)

made hotpixel mask
loaded bias
Processing the following files:
0 m82_h-alpha_128.0s_bin1L_220124_084013_al_seo_0_RAW.fits
0 m82_h-alpha_128.0s_bin1H_220124_084013_al_seo_0_RAW.fits
1 m82_h-alpha_128.0s_bin1L_220124_084243_al_seo_1_RAW.fits
1 m82_h-alpha_128.0s_bin1H_220124_084243_al_seo_1_RAW.fits
2 m82_h-alpha_128.0s_bin1L_220124_084513_al_seo_2_RAW.fits
2 m82_h-alpha_128.0s_bin1H_220124_084513_al_seo_2_RAW.fits
3 m82_h-alpha_128.0s_bin1L_220124_084743_al_seo_3_RAW.fits
3 m82_h-alpha_128.0s_bin1H_220124_084743_al_seo_3_RAW.fits
4 m82_h-alpha_128.0s_bin1L_220124_085013_al_seo_4_RAW.fits
4 m82_h-alpha_128.0s_bin1H_220124_085013_al_seo_4_RAW.fits
5 m82_h-alpha_128.0s_bin1L_220124_085244_al_seo_5_RAW.fits
5 m82_h-alpha_128.0s_bin1H_220124_085244_al_seo_5_RAW.fits
6 m82_h-alpha_128.0s_bin1L_220124_085514_al_seo_6_RAW.fits
6 m82_h-alpha_128.0s_bin1H_220124_085514_al_seo_6_RAW.fits
7 m82_h-alpha_128.0s_bin1L_220124_085744_al_seo_7_RAW.fits
7 m82_h-alpha_128.0s_bin1H_220124_085744_al_seo_7_RA

<Figure size 1296x576 with 0 Axes>

In [10]:
"""
run_pipeline:
    Runs the pipeline with the hotpix and astrometry steps
    on all the files in the directory.

    Inputs:
    folder (str): The folder containing the files to be processed. (Default: outpath from setup.py)
    Note: Only HDR images will be processed by the pipeline

    Outputs:
    Saves HPX and WPS files in the outpath.

    NOTE: I Recommend looking through all the HDR files in the folder and removing
    any with major noticeable issues, such as lost tracking or closed slit, as these
    will make Astrometry take much longer
"""
#run_pipeline(folder = out)
run_local_astrometry(folder = out)


Running local astrometry on the following files:
m82_h-alpha_128.0s_bin1_220124_084013_al_seo_0_HDR.fits
m82_h-alpha_128.0s_bin1_220124_084243_al_seo_1_HDR.fits
m82_h-alpha_128.0s_bin1_220124_084513_al_seo_2_HDR.fits
m82_h-alpha_128.0s_bin1_220124_084743_al_seo_3_HDR.fits
m82_h-alpha_128.0s_bin1_220124_085013_al_seo_4_HDR.fits
m82_h-alpha_128.0s_bin1_220124_085244_al_seo_5_HDR.fits
m82_h-alpha_128.0s_bin1_220124_085514_al_seo_6_HDR.fits
m82_h-alpha_128.0s_bin1_220124_085744_al_seo_7_HDR.fits
m82_h-alpha_128.0s_bin1_220124_090015_al_seo_8_HDR.fits
m82_h-alpha_128.0s_bin1_220124_090245_al_seo_9_HDR.fits
m82_h-alpha_128.0s_bin1_220124_090515_al_seo_10_HDR.fits
m82_h-alpha_128.0s_bin1_220124_090746_al_seo_11_HDR.fits
m82_h-alpha_128.0s_bin1_220124_091016_al_seo_12_HDR.fits
m82_h-alpha_128.0s_bin1_220124_091246_al_seo_13_HDR.fits
m82_h-alpha_128.0s_bin1_220124_091517_al_seo_14_HDR.fits
m82_h-alpha_128.0s_bin1_220124_091747_al_seo_15_HDR.fits
m82_h-alpha_128.0s_bin1_220124_092402_al_seo_16_H

In [17]:
"""
drizzle:
    Drizzles all WCS images in the output folder with given filters
    inputs: 
        - filter (optional lst str): list of filter strings to be drizzled. Will combine all images containing each filter into a single image.
        If not specified, will combine all images in the input folder
        - infolder (str): path to folder containing images to stack (default: outpath set in setup.py)
        Note: Infolder is set to outpath by default so it can take the output of the previous steps
        - outfolder (str): path to output folder (default: outpath set in setup.py)
    outputs:
        - Saves drizzled images to outpath
    example:
        drizzle(filter=['g-band','i-band'], infolder = out, outfolder = out) drizzles all images containing the string "g-band into their respective files
"""
drizzle(filter = ['WCS.fits'], infolder = out, outfolder = out)

GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration


['m82_h-alpha_128.0s_bin1_220124_101004_al_seo_32_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_084743_al_seo_3_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_100504_al_seo_30_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_095502_al_seo_26_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_095232_al_seo_25_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_092402_al_seo_16_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_084513_al_seo_2_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_085013_al_seo_4_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_091246_al_seo_13_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_091517_al_seo_14_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_085244_al_seo_5_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_101504_al_seo_34_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_095733_al_seo_27_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_100234_al_seo_29_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_090746_al_seo_11_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_102005_al_seo_36_WCS.fits', 'm82_h-alpha_128.0s_bin1_220124_092634_al_s

GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Mi

Data loaded
Starting drizzle
0 Adding /Users/jonahdf/Local Documents/Astro/M82 1:28/m82-cosmic-test/m82_h-alpha_128.0s_bin1_220124_084013_al_seo_0_WCS.fits to drizzle stack
1 Adding /Users/jonahdf/Local Documents/Astro/M82 1:28/m82-cosmic-test/m82_h-alpha_128.0s_bin1_220124_084243_al_seo_1_WCS.fits to drizzle stack
2 Adding /Users/jonahdf/Local Documents/Astro/M82 1:28/m82-cosmic-test/m82_h-alpha_128.0s_bin1_220124_084513_al_seo_2_WCS.fits to drizzle stack
3 Adding /Users/jonahdf/Local Documents/Astro/M82 1:28/m82-cosmic-test/m82_h-alpha_128.0s_bin1_220124_084743_al_seo_3_WCS.fits to drizzle stack
4 Adding /Users/jonahdf/Local Documents/Astro/M82 1:28/m82-cosmic-test/m82_h-alpha_128.0s_bin1_220124_085013_al_seo_4_WCS.fits to drizzle stack
5 Adding /Users/jonahdf/Local Documents/Astro/M82 1:28/m82-cosmic-test/m82_h-alpha_128.0s_bin1_220124_085244_al_seo_5_WCS.fits to drizzle stack
6 Adding /Users/jonahdf/Local Documents/Astro/M82 1:28/m82-cosmic-test/m82_h-alpha_128.0s_bin1_220124_08551

In [14]:
#files = ['/Users/jonahdf/Local Documents/Astro/M82 1:28/output/red.fits', '/Users/jonahdf/Local Documents/Astro/M82 1:28/output/blue.fits', '/Users/jonahdf/Local Documents/Astro/M82 1:28/output/green.fits']
# get all files in directory
files = [os.path.join(out,file) for file in os.listdir(out) if 'WCS.f' in file]
filesFits = [DataFits(file) for file in files]
def cosmic_file(a):
    return detect_cosmics(a.image, cleantype='medmask')

removed_rays = map(cosmic_file, filesFits)
for x, y in zip(removed_rays, filesFits):
    y.image = x[1]
    y.filename = y.filename.replace('.fits', '_CR.fits')
    y.save()

GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Mi

In [ ]:
display_images(out, "HDR")

In [24]:
newFiles = [os.path.join(out,file) for file in os.listdir(out) if 'WCS.f' in file]
newFilesFits = [DataFits(file) for file in newFiles]


GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Missing Configuration
GetHeadVal: Mi